# two base tracks away

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('DS_2_train.csv', index_col=0)
df_test = pd.read_csv('DS_2_test.csv', index_col=0)

/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
zl = np.unique(df['Z'])
zmin = np.min(zl)
zmax = np.max(zl)
dz = np.diff(zl)[0]

In [4]:
def dmin(v, vl):
    return np.min(np.sum((v.values - vl)**2, axis=1))
def ddmin_f(efr, df, n):
    z0 = efr.iloc[0]['Z']
    if z0 >= zmax - (n-1)*dz:
        efr['dfore'] = 1e6
    else:
        eto = df.query("Z=={}".format(z0 + n*dz))
        gfr = efr[["X1", "Y1", "X2", "Y2"]]
        gto = eto[["X", "Y", "XE", "YE"]]
        d = gfr.apply(dmin, axis=1, args=(gto.values,))
        efr['dfore'] = d
    return efr
def ddmin_b(eto, df, n):
    z0 = eto.iloc[0]['Z']
    if z0 <= zmin + (n-1)*dz:
        eto['dback'] = 1e6
    else:
        efr = df.query("Z=={}".format(z0 - n*dz))
        gfr = efr[["X1", "Y1", "X2", "Y2"]]
        gto = eto[["X", "Y", "XE", "YE"]]
        d = gto.apply(dmin, axis=1, args=(gfr.values,))
        eto['dback'] = d
    return eto
def ddmin(ev, df, n):
    return ddmin_b(ddmin_f(ev, df, n), df, n)

In [5]:
def features(df, n):
    dx = df['TX'] * dz
    dy = df['TY'] * dz
    df['XE'] = df['X'] + dx
    df['YE'] = df['Y'] + dy
    df['X1'] = df['X'] + (n-1) * dx
    df['Y1'] = df['Y'] + (n-1) * dy
    df['X2'] = df['X1'] + n * dx
    df['Y2'] = df['Y1'] + n * dy
    return df.groupby("Z").apply(ddmin, df, n)

In [6]:
def base_n_away(n):
    rename = {"dfore":"dfore{}".format(n), "dback":"dback{}".format(n)}
    xdf = df.groupby("brick_number").apply(features, n)
    xdfo = xdf[['dfore', 'dback']].rename(columns=rename)
    xdfo.to_hdf("data-r{}.h5".format(n), "train")
    xdft = df_test.groupby("brick_number").apply(features, n)
    xdfot = xdft[['dfore', 'dback']].rename(columns=rename)
    xdfot.to_hdf("data-r{}.h5".format(n), "test")
    return n

In [7]:
from concurrent.futures import ProcessPoolExecutor

In [8]:
with ProcessPoolExecutor(max_workers=5) as executor:
    rst = list(executor.map(base_n_away, (2,3,4,5,6)))